In [11]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

In [12]:
# Load the datasets
awards_players_df = pd.read_csv('../../dataset/awards_players.csv')
coaches_df = pd.read_csv('../../dataset/coaches.csv')
players_df = pd.read_csv('../../dataset/players.csv')
players_teams_df = pd.read_csv('../../dataset/players_teams.csv')
series_post_df = pd.read_csv('../../dataset/series_post.csv')
teams_df = pd.read_csv('../../dataset/teams.csv')
teams_post_df = pd.read_csv('../../dataset/teams_post.csv')

In [13]:
# Remove useless columns from the datasets
awards_players_df = awards_players_df.drop(columns=['lgID'])
players_df = players_df.drop(columns=['firstseason', 'lastseason', 'deathDate'])
coaches_df = coaches_df.drop(columns=['lgID'])
series_post_df = series_post_df.drop(columns=['lgIDLoser', 'lgIDWinner'])
teams_post_df = teams_post_df.drop(columns=['lgID'])
teams_df = teams_df.drop(columns=['lgID', 'divID', 'seeded', 'tmORB', 'tmDRB', 'tmTRB', 'opptmORB', 'opptmDRB', 'opptmTRB'])
players_teams_df = players_teams_df.drop(columns=['lgID'])

In [28]:
# Display the first few rows of each cleaned DataFrame (optional)
awards_players_df.head()

,playerID,award,year
0,thompti01w,All-Star Game Most Valuable Player,1
1,leslili01w,All-Star Game Most Valuable Player,2
2,leslili01w,All-Star Game Most Valuable Player,3
3,teaslni01w,All-Star Game Most Valuable Player,4
4,swoopsh01w,All-Star Game Most Valuable Player,6


In [29]:
players_df.head()

,bioID,pos,height,weight,college,collegeOther,birthDate
0,abrahta01w,C,74.0,190,George Washington,NaN,1975-09-27
1,abrossv01w,F,74.0,169,Connecticut,NaN,1980-07-09
2,adairje01w,C,76.0,197,George Washington,NaN,1986-12-19
3,adamsda01w,F-C,73.0,239,Texas A&M,Jefferson College (JC),1989-02-19
4,adamsjo01w,C,75.0,180,New Mexico,NaN,1981-05-24


In [30]:
coaches_df.head()

,coachID,year,tmID,stint,won,lost,post_wins,post_losses
0,adamsmi01w,5,WAS,0,17,17,1,2
1,adubari99w,1,NYL,0,20,12,4,3
2,adubari99w,2,NYL,0,21,11,3,3
3,adubari99w,3,NYL,0,18,14,4,4
4,adubari99w,4,NYL,0,16,18,0,0


In [31]:
series_post_df.head()

,year,round,series,tmIDWinner,tmIDLoser,W,L
0,1,FR,A,CLE,ORL,2,1
1,1,FR,B,NYL,WAS,2,0
2,1,FR,C,LAS,PHO,2,0
3,1,FR,D,HOU,SAC,2,0
4,1,CF,E,HOU,LAS,2,0


In [32]:
teams_post_df.head()

,year,tmID,W,L
0,1,HOU,6,0
1,1,ORL,1,2
2,1,CLE,3,3
3,1,WAS,0,2
4,1,NYL,4,3


In [33]:
teams_df.head()

,year,tmID,franchID,confID,rank,playoff,firstRound,semis,finals,name,...,homeW,homeL,awayW,awayL,confW,confL,min,attend,arena,playoffNextYear
0,9,ATL,ATL,EA,7,N,NaN,NaN,NaN,Atlanta Dream,...,1,16,3,14,2,18,6825,141379,Philips Arena,1.0
1,10,ATL,ATL,EA,2,Y,L,NaN,NaN,Atlanta Dream,...,12,5,6,11,10,12,6950,120737,Philips Arena,0.0
2,1,CHA,CHA,EA,8,N,NaN,NaN,NaN,Charlotte Sting,...,5,11,3,13,5,16,6475,90963,Charlotte Coliseum,1.0
3,2,CHA,CHA,EA,4,Y,W,W,L,Charlotte Sting,...,11,5,7,9,15,6,6500,105525,Charlotte Coliseum,1.0
4,3,CHA,CHA,EA,2,Y,L,NaN,NaN,Charlotte Sting,...,11,5,7,9,12,9,6450,106670,Charlotte Coliseum,1.0


In [34]:
players_teams_df.head()

,playerID,year,stint,tmID,GP,GS,minutes,points,oRebounds,dRebounds,...,PostBlocks,PostTurnovers,PostPF,PostfgAttempted,PostfgMade,PostftAttempted,PostftMade,PostthreeAttempted,PostthreeMade,PostDQ
0,abrossv01w,2,0,MIN,26,23,846,343,43,131,...,0,0,0,0,0,0,0,0,0,0
1,abrossv01w,3,0,MIN,27,27,805,314,45,101,...,0,0,0,0,0,0,0,0,0,0
2,abrossv01w,4,0,MIN,30,25,792,318,44,97,...,1,8,8,22,6,8,8,7,3,0
3,abrossv01w,5,0,MIN,22,11,462,146,17,57,...,2,3,7,23,8,4,2,8,2,0
4,abrossv01w,6,0,MIN,31,31,777,304,29,78,...,0,0,0,0,0,0,0,0,0,0


In [35]:
# Merge datasets
players_teams_merged = pd.merge(players_df, players_teams_df, left_on='bioID', right_on='playerID')

team_player_stats = players_teams_merged.groupby('tmID').agg({
    'points': 'mean',
    'assists': 'mean',
    'rebounds': 'mean'
}).reset_index()


teams_df = teams_df.sort_values(['tmID', 'year'])
teams_df['playoffNextYear'] = teams_df['playoff'].shift(-1)
teams_df.loc[teams_df['franchID'] != teams_df['franchID'].shift(-1), 'playoffNextYear'] = None
teams_df['playoffNextYear'] = teams_df['playoffNextYear'].map({'Y': 1, 'N': 0})
teams_df['playoffNextYear'] = teams_df['playoffNextYear'].fillna(0)

teams_full = pd.merge(teams_df, team_player_stats, on='tmID')

features = ['points', 'assists', 'rebounds']
target = 'playoffNextYear'

In [ ]:
teams_full.head()

In [26]:
train_data = teams_full[teams_full['year'] < 5]
test_data = teams_full[teams_full['year'].isin([6,7])]

X_train = train_data[features]
y_train = train_data[target]
X_test = test_data[features]
y_test = test_data[target]

In [27]:
model = DecisionTreeClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print('Classification Report:\n', report)

Accuracy: 0.4444444444444444
Classification Report:
               precision    recall  f1-score   support

         0.0       0.36      0.45      0.40        11
         1.0       0.54      0.44      0.48        16

    accuracy                           0.44        27
   macro avg       0.45      0.45      0.44        27
weighted avg       0.46      0.44      0.45        27

